In [ ]:
!pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.2 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23777 sha256=86c63ade45cd34b5d3f824d3771a318f32f4dfd96744e0a61827194682327855
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=0289260530fbfecb195408483def54c30161fa03c65c2

In [ ]:
#Load KeyBERT libraries
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import nltk
import re
import heapq
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
#Load the data
df = pd.read_csv('covid19.csv')
df.head()

"""
#Preprocess the data
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

df['Abstract'] = df['Abstract'].apply(preprocess)
df.head()
"""
#Extract keywords using KeyBERT
model = KeyBERT('distilbert-base-nli-mean-tokens')
keywords = model.extract_keywords(df['Abstract'][0], keyphrase_ngram_range=(1, 1),
                                  stop_words='english', use_maxsum=True, nr_candidates=20, top_n=5)
keywords

[('viral', 0.1693),
 ('symptomatic', 0.2165),
 ('pcr', 0.2376),
 ('radiologists', 0.2734),
 ('pneumonia', 0.3525)]

In [ ]:
#Text Summarization using ClinicalBERT
#load the libraries
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
#load the model
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModelWithLMHead.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
#Tokenize the text
text = df['Abstract'][0]
tokenized_text = tokenizer.tokenize(text)
#Convert tokens to ids
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
#Convert ids to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
#Summarize the text
summary_ids = model.generate(tokens_tensor, max_length=402, min_length=50, length_penalty=5, num_beams=2, no_repeat_ngram_size=2, early_stopping=True)
#Convert the ids to text
summary = tokenizer.decode(summary_ids[0])
print(summary)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


objective : to review the available literature concerning the effectiveness of the covid - 19 diagnostic tools. background : with the absence of specific treatment / vaccines for the coronavirus covid - 19, the most appropriate approach to control this infection is to quarantine people and isolate symptomatic people and suspected or infected cases. although real - time reverse transcription - polymerase chain reaction ( rt - pcr ) assay is considered the first tool to make a definitive diagnosis of covid - 19 disease, the high false negative rate, low sensitivity, limited supplies and strict requirements for laboratory settings might delay accurate diagnosis. computed tomography ( ct ) has been reported as an important tool to identify and investigate suspected patients with covid - 19 disease at early stage. key findings : rt - pcr shows low sensitivity ( 60 - 71 % ) in diagnosing patients with covid - 19 infection compared to the ct chest. several studies reported that chest ct scans

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install bert-extractive-summarizer
!pip install rouge

In [ ]:
#evaluate the summary's quality
from rouge import Rouge
rouge = Rouge()
scores = rouge.get_scores(summary, df['Abstract'][0])
scores

[{'rouge-1': {'r': 0.8238993710691824,
   'p': 0.7891566265060241,
   'f': 0.8061538411561657},
  'rouge-2': {'r': 0.6965811965811965,
   'p': 0.6150943396226415,
   'f': 0.65330660824575},
  'rouge-l': {'r': 0.8238993710691824,
   'p': 0.7891566265060241,
   'f': 0.8061538411561657}}]

In [ ]:
#Pruning is a technique that allows you to reduce the size of a model by removing unnecessary parameters.
import torch
import torch.nn.utils.prune as prune
#prune the model to reduce the size of the model
def pruning_model(model, pruning_rate):
    parameters_to_prune =[]
    for name, module in model.named_modules():
        #Bert Attention Layer Pruning
        if isinstance(module, torch.nn.MultiheadAttention):
            parameters_to_prune.append((module, 'in_proj_weight'))
            parameters_to_prune.append((module, 'out_proj.weight'))
            parameters_to_prune.append((module, 'out_proj.bias'))
        #Bert Embedding Layer Pruning
        elif isinstance(module, torch.nn.Embedding):
            parameters_to_prune.append((module, 'weight'))
        #Bert LayerNorm Layer Pruning
        elif isinstance(module, torch.nn.LayerNorm):
            parameters_to_prune.append((module, 'weight'))
            parameters_to_prune.append((module, 'bias'))
        #Bert Linear Layer Pruning
        elif isinstance(module, torch.nn.Linear):
            parameters_to_prune.append((module, 'weight'))
            parameters_to_prune.append((module, 'bias'))
    parameters_to_prune = tuple(parameters_to_prune)
    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=pruning_rate,
    )

    #start pruning
    print(prune.is_pruned(model))
    prune.remove(model, 'weight')

    #return the pruned model
    return model



In [ ]:
#BioClinicalBERT is a pre-trained language model for clinical text.
#describe the model's layers
model_layers = model.named_modules()
model_layers = list(model_layers)
print(model_layers)

[('', BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [ ]:
pruned_model = pruning_model(model,0.3)

True


ValueError: Parameter 'weight' of module BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [ ]:
#compare the model size before and after pruning by the number of parameters
print("Number of parameters before pruning: ", sum(p.numel() for p in model.parameters()))
print("Number of parameters after pruning: ", sum(p.numel() for p in pruned_model.parameters()))

In [ ]:
text = df['Abstract'][0]
tokenized_text = tokenizer.tokenize(text)
#Convert tokens to ids
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
#Convert ids to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
#Summarize the text
summary_ids = pruned_model.generate(tokens_tensor, max_length=402, min_length=50, length_penalty=5, num_beams=2, no_repeat_ngram_size=2, early_stopping=True)
#Convert the ids to text
summary = tokenizer.decode(summary_ids[0])
print(summary)


In [ ]:
rouge = Rouge()
scores = rouge.get_scores(summary, df['Abstract'][0])
scores

# Explore Alternative

In [ ]:
#load BertSum from huggingface transformers
from transformers import BertModel, BertTokenizer , AutoModel , pipeline
model = AutoModel.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext')
tokenizer = AutoTokenizer.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext')

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model_bio = AutoModel.from_pretrained('monologg/biobert_v1.1_pubmed')
tokenizer_bio = AutoTokenizer.from_pretrained('monologg/biobert_v1.1_pubmed')

Some weights of the model checkpoint at monologg/biobert_v1.1_pubmed were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
summarizer=pipeline('summarization',model=model,tokenizer=tokenizer)

The model 'BertModel' is not supported for summarization. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


In [ ]:
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

#run on a sample text
text="The COVID-19 pandemic, also known as the coronavirus pandemic, is an ongoing pandemic of coronavirus disease 2019 (COVID-19) caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). It was first identified in December 2019 in Wuhan, China. The World Health Organization declared the outbreak a Public Health Emergency of International Concern on 30 January 2020 and a pandemic on 11 March. As of 26 April 2021, more than 147 million cases have been confirmed, with more than 3.11 million deaths attributed to COVID-19, making it one of the deadliest pandemics in history."

#summarize the text
summary=summarizer(text, max_length=100, min_length=5, do_sample=False)[0]['summary_text']

The model 'BertModel' is not supported for summarization. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


TypeError: The current model class (BertModel) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'BertLMHeadModel'}

In [ ]:
!pip install bert-extractive-summarizer

In [ ]:
from summarizer import Summarizer
model = Summarizer()
result = model(df['Abstract'][0], min_length=20)
summary = "".join(result)
print(summary)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Objective:                    To review the available literature concerning the effectiveness of the COVID-19 diagnostic tools. Although real-time reverse transcription-polymerase chain reaction (RT-PCR) assay is considered the first tool to make a definitive diagnosis of COVID-19 disease, the high false negative rate, low sensitivity, limited supplies and strict requirements for laboratory settings might delay accurate diagnosis. Conclusion:                    Although CT scan is a powerful tool in COVID-19 diagnosis, it is not sufficient to detect COVID-19 alone due to the low specificity (25%), and challenges that radiologists might face in differentiating COVID-19 from other viral pneumonia on chest CT scans.


In [ ]:
rouge = Rouge()
scores = rouge.get_scores(summary, df['Abstract'][0])
scores

[{'rouge-1': {'r': 0.4591194968553459, 'p': 1.0, 'f': 0.6293103405146403},
  'rouge-2': {'r': 0.3974358974358974,
   'p': 0.9789473684210527,
   'f': 0.5653495399654475},
  'rouge-l': {'r': 0.4591194968553459, 'p': 1.0, 'f': 0.6293103405146403}}]

In [ ]:
#finetune the model
#finetune the model using pubmed data
#load the dataset from huggingface
from datasets import load_dataset
dataset = load_dataset("ccdv/pubmed-summarization", split="train")

#Train the model
#load the trainer
from transformers import Trainer, TrainingArguments
#load the training arguments
training_args = TrainingArguments(
    output_dir="./longt5-tglobal-large-16384-pubmed-3k_steps",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    save_steps=1000,
    save_total_limit=1,
    prediction_loss_only=True,
    learning_rate=2e-4,
    weight_decay=0.01,
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
    warmup_steps=100,
    gradient_accumulation_steps=16,
    fp16=True,
    fp16_opt_level="O1",
    max_grad_norm=1.0,
    seed=42,

)

#load the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

#Set to train on GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

#train the model
trainer.train()

# Frankstein

In [ ]:
!pip install accelerate

In [ ]:
pip install bitsandbytes

In [ ]:
!pip install transformers --upgrade

In [ ]:
#Text Generation with GPT-2
#GPT-2 is a transformer-based language model that uses the attention mechanism to generate text.
#It is a powerful model that can be used to generate text for any task.
#Quantize the model to 8-bit integer
#load the model
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

#Text Generation pipeline with GPT-2
#load the pipeline
from transformers import pipeline
generator = pipeline('text-generation', model='gpt2')

In [ ]:
#prompt the model to summarize the text
output=generator("Summarize the text: "+text, max_length=1000, num_return_sequences=5)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
print(text)

Objective:                    To review the available literature concerning the effectiveness of the COVID-19 diagnostic tools.              Background:                    With the absence of specific treatment/vaccines for the coronavirus COVID-19, the most appropriate approach to control this infection is to quarantine people and isolate symptomatic people and suspected or infected cases. Although real-time reverse transcription-polymerase chain reaction (RT-PCR) assay is considered the first tool to make a definitive diagnosis of COVID-19 disease, the high false negative rate, low sensitivity, limited supplies and strict requirements for laboratory settings might delay accurate diagnosis. Computed tomography (CT) has been reported as an important tool to identify and investigate suspected patients with COVID-19 disease at early stage.              Key findings:                    RT-PCR shows low sensitivity (60-71%) in diagnosing patients with COVID-19 infection compared to the CT 

In [ ]:
output[2]

{'generated_text': "Summarize the text: Objective:                    To review the available literature concerning the effectiveness of the COVID-19 diagnostic tools.              Background:                    With the absence of specific treatment/vaccines for the coronavirus COVID-19, the most appropriate approach to control this infection is to quarantine people and isolate symptomatic people and suspected or infected cases. Although real-time reverse transcription-polymerase chain reaction (RT-PCR) assay is considered the first tool to make a definitive diagnosis of COVID-19 disease, the high false negative rate, low sensitivity, limited supplies and strict requirements for laboratory settings might delay accurate diagnosis. Computed tomography (CT) has been reported as an important tool to identify and investigate suspected patients with COVID-19 disease at early stage.              Key findings:                    RT-PCR shows low sensitivity (60-71%) in diagnosing patients wit

In [ ]:
prompt="Summarize the text"
#Remove the prompt and the text from the output
output_test = output[1]['generated_text'].replace(prompt, "")
output_test = output_test.replace(text, "")

#Rouge results
rouge = Rouge()
rouge_results = rouge.get_scores(output_test, text)
print(rouge_results)

[{'rouge-1': {'r': 0.27044025157232704, 'p': 0.23118279569892472, 'f': 0.2492753573494645}, 'rouge-2': {'r': 0.05128205128205128, 'p': 0.04054054054054054, 'f': 0.045283013936347996}, 'rouge-l': {'r': 0.22641509433962265, 'p': 0.1935483870967742, 'f': 0.20869564720453698}}]


In [ ]:
#Summarization-specific
from transformers import AutoModel,AutoTokenizer
model_sum = AutoModel.from_pretrained("Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")
tokenizer_sum = AutoTokenizer.from_pretrained("Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")

In [ ]:
finetune_df=pd.read_csv('DATACovid - covid19.csv')

In [ ]:
#finetune the model
#finetune the model using pubmed data
#load the dataset from huggingface
from datasets import load_dataset
dataset = load_dataset("ccdv/pubmed-summarization", split="train")

#Train the model
#load the trainer
from transformers import Trainer, TrainingArguments
#load the training arguments
training_args = TrainingArguments(
    output_dir="./longt5-tglobal-large-16384-pubmed-3k_steps",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    save_steps=1000,
    save_total_limit=1,
    prediction_loss_only=True,
    learning_rate=2e-4,
    weight_decay=0.01,
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
    warmup_steps=100,
    gradient_accumulation_steps=16,
    fp16=True,
    fp16_opt_level="O1",
    max_grad_norm=1.0,
    seed=42,

)

#load the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

#train the model
trainer.train()
